<a href="https://colab.research.google.com/github/rodrigozago/compiladores/blob/master/Lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analisador léxico
==================

Definição da tabela de simbolos
--------------------

Classe que define a tabela de simbolos.

symTable: Estrutura do tipo dict para armazenar os tokens:

```{ lexema<string> : token<Token> }```

In [253]:
class SymTable:
    def __init__(self):
        self.symTable = dict();
    def addNewEntry(self, token):
        if(token.lexema not in self.symTable):
            self.symTable[token.lexema] = token
        return token
    def getEntry(self, lexema):
        if(lexema in self.symTable):
            return self.symTable[lexema]
        else:
            return None

Definição da classe Token
--------------------------

In [216]:
class Token:
  def __init__(self, lexema, token, tipo):
    self.lexema = lexema
    self.token = token
    self.tipo = tipo

  def __str__(self):
    return "Lexema: {} Token: {} Tipo: {}".format(self.lexema, self.token, self.tipo)

Adicionando palavras chave da linguagem na tabela de simbolos
---------------------------------------

In [217]:
# Creating symbol table
table = SymTable()

# Add keywords into symbol table
try:
    table.addNewEntry(Token('inicio', 'inicio', '-'))
    table.addNewEntry(Token('varinicio', 'varinicio', '-'))
    table.addNewEntry(Token('varfim', 'varfim', '-'))
    table.addNewEntry(Token('escreva', 'escreva', '-'))
    table.addNewEntry(Token('leia', 'leia', '-'))
    table.addNewEntry(Token('se', 'se', '-'))
    table.addNewEntry(Token('entao', 'entao', '-'))
    table.addNewEntry(Token('fimse', 'fimse', '-'))
    table.addNewEntry(Token('fim', 'fim', '-'))
    table.addNewEntry(Token('inteiro', 'inteiro', '-'))
    table.addNewEntry(Token('lit', 'lit', '-'))
    table.addNewEntry(Token('real', 'real', '-'))    
except Exception as e:
    print("erro: {}".format(e))



debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando
debug: adicionando


In [218]:
print(table.addNewEntry(Token('inicio', 'inicio', '-')))

Lexema: inicio Token: inicio Tipo: -


In [229]:
print(table.symTable['inicio'])

Lexema: inicio Token: inicio Tipo: -


Tabela dos estados finais
-------------

In [ ]:
state_tokens = {
    1: 'Num',
    2: 'Num',
    3: 'Num',
    6: 'Num',
    8: 'Literal',
    9: 'id',
    11: 'Comentário',
    12: 'EOF',
    13: 'OPR',
    14: 'OPR',
    15: 'OPR',
    16: 'OPR',
    17: 'RCB',
    18: 'OPR',
    19: 'OPM',
    20: 'AB_P',
    21: 'FC_P',
    22: 'PTV'
}

Definição do autômato finito
-----------------------------

Definicao da class DFA

In [246]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        if((self.current_state, input_value) not in self.transition_function.keys()):
            
            if((self.current_state, '@') in self.transition_function.keys()):
                
                self.current_state = self.transition_function[(self.current_state, '@')];
                return True;
            
            return False;
        
        previous_state = self.current_state;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        
        print("({}, {}) -> {}".format(previous_state, input_value, self.current_state))
        
        return True;
    
    def analisys(self, input_list):
        self.go_to_initial_state();
        tokens = []
        lexema = ""
        last_accept_state = None
        i = 0

        while i < len(input_list): 
            thisState = self.current_state
            didTransition = self.transition_to_state_with_input(input_list[i]);
            
            if(self.current_state is 23):
                i = i + 1
                self.go_to_initial_state()
                lexema = ""
                continue
            
            if didTransition:
                lexema = lexema + input_list[i]
                last_accept_state = self.current_state
                if i is len(input_list)-1:
                    token = Token(lexema, state_tokens[self.current_state], '-')
                    print(token)
                    tokens.append(token)
                    self.go_to_initial_state()
                    lexema = ""
            else:
                if self.in_accept_state():
                    token = Token(lexema, state_tokens[self.current_state], '-')
                    token = self.addToSymbolTable(lexema, token)
                    print(token)
                    tokens.append(token)
                    self.go_to_initial_state()
                    lexema = ""
                    continue
                else: 
                    print("Erro em: ({}, {}) -> {}".format('?', input_list[i], self.current_state))
                    lexema = ""
                    self.go_to_initial_state()
                    
            i = i + 1
            
            
    def addToSymbolTable(self, lexema, token):
        result = table.getEntry(lexema)
        if result:
            return result
        else: 
            return table.addNewEntry(token)
        
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;
    
    def run_with_input_list(self, input_list):
        self.go_to_initial_state();
        for inp in input_list:
            self.transition_to_state_with_input(inp);
            continue;
        return self.in_accept_state();

Definindo função de transição

In [247]:
tf = dict();

# Ignore tab, \n, and spaces
# tf[(0, ' ')] = 0;
# tf[(0, '\n')] = 0;
# tf[(0, '\t')] = 0;

tf[(0, ' ')] = 23;
tf[(0, '\n')] = 23;
tf[(0, '\t')] = 23;

# Begin tf[(0, D)] = 1;
tf[(0, '0')] = 1;
tf[(0, '1')] = 1;
tf[(0, '2')] = 1;
tf[(0, '3')] = 1;
tf[(0, '4')] = 1;
tf[(0, '5')] = 1;
tf[(0, '6')] = 1;
tf[(0, '7')] = 1;
tf[(0, '8')] = 1;
tf[(0, '9')] = 1;
# End tf[(0, D)] = 1;

# Begin tf[(1, D)] = 1;
tf[(1, '0')] = 1;
tf[(1, '1')] = 1;
tf[(1, '2')] = 1;
tf[(1, '3')] = 1;
tf[(1, '4')] = 1;
tf[(1, '5')] = 1;
tf[(1, '6')] = 1;
tf[(1, '7')] = 1;
tf[(1, '8')] = 1;
tf[(1, '9')] = 1;
# End tf[(1, D)] = 1;

#TODO RESOLVER \. 
tf[(1, '.')] = 2;
tf[(1, 'E')] = 4;
tf[(1, 'e')] = 4;

# Begin tf[(2, D)] = 3;
tf[(2, '0')] = 3;
tf[(2, '1')] = 3;
tf[(2, '2')] = 3;
tf[(2, '3')] = 3;
tf[(2, '4')] = 3;
tf[(2, '5')] = 3;
tf[(2, '6')] = 3;
tf[(2, '7')] = 3;
tf[(2, '8')] = 3;
tf[(2, '9')] = 3;
# End tf[(2, D)] = 3;

# TODO VERIFICAR COM O PRATES
# tf[(2, "QUALQUER COISA")] = 23;

# Begin tf[(3, D)] = 3;
tf[(3, '0')] = 3;
tf[(3, '1')] = 3;
tf[(3, '2')] = 3;
tf[(3, '3')] = 3;
tf[(3, '4')] = 3;
tf[(3, '5')] = 3;
tf[(3, '6')] = 3;
tf[(3, '7')] = 3;
tf[(3, '8')] = 3;
tf[(3, '9')] = 3;
# End tf[(3, D)] = 3;

tf[(3, 'E')] = 4;
tf[(3, 'e')] = 4;
tf[(4, '+')] = 5;
tf[(4, '-')] = 5;

# Begin tf[(4, D)] = 6;
tf[(4, '0')] = 6;
tf[(4, '1')] = 6;
tf[(4, '2')] = 6;
tf[(4, '3')] = 6;
tf[(4, '4')] = 6;
tf[(4, '5')] = 6;
tf[(4, '6')] = 6;
tf[(4, '7')] = 6;
tf[(4, '8')] = 6;
tf[(4, '9')] = 6;
# End tf[(4, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(4, 'QUALQUER COISA')] = 23;

# Begin tf[(5, D)] = 6;
tf[(5, '0')] = 6;
tf[(5, '1')] = 6;
tf[(5, '2')] = 6;
tf[(5, '3')] = 6;
tf[(5, '4')] = 6;
tf[(5, '5')] = 6;
tf[(5, '6')] = 6;
tf[(5, '7')] = 6;
tf[(5, '8')] = 6;
tf[(5, '9')] = 6;
# End tf[(5, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(5, 'QUALQUER COISA')] = 23;

# Begin tf[(6, D)] = 6;
tf[(6, '0')] = 6;
tf[(6, '1')] = 6;
tf[(6, '2')] = 6;
tf[(6, '3')] = 6;
tf[(6, '4')] = 6;
tf[(6, '5')] = 6;
tf[(6, '6')] = 6;
tf[(6, '7')] = 6;
tf[(6, '8')] = 6;
tf[(6, '9')] = 6;
# End tf[(6, D)] = 6;

tf[(0, '"')] = 7;
tf[(7, '@')] = 7; # aqui o @ substitui ., aceita tudo 
tf[(7, '"')] = 8;

# Begin tf[(0, L)] = 9;
tf[(0, 'a')] = 9;
tf[(0, 'b')] = 9;
tf[(0, 'c')] = 9;
tf[(0, 'd')] = 9;
tf[(0, 'e')] = 9;
tf[(0, 'f')] = 9;
tf[(0, 'g')] = 9;
tf[(0, 'h')] = 9;
tf[(0, 'i')] = 9;
tf[(0, 'j')] = 9;
tf[(0, 'k')] = 9;
tf[(0, 'l')] = 9;
tf[(0, 'm')] = 9;
tf[(0, 'n')] = 9;
tf[(0, 'o')] = 9;
tf[(0, 'p')] = 9;
tf[(0, 'q')] = 9;
tf[(0, 'r')] = 9;
tf[(0, 's')] = 9;
tf[(0, 't')] = 9;
tf[(0, 'u')] = 9;
tf[(0, 'v')] = 9;
tf[(0, 'w')] = 9;
tf[(0, 'x')] = 9;
tf[(0, 'y')] = 9;
tf[(0, 'z')] = 9;
tf[(0, 'A')] = 9;
tf[(0, 'B')] = 9;
tf[(0, 'C')] = 9;
tf[(0, 'D')] = 9;
tf[(0, 'E')] = 9;
tf[(0, 'F')] = 9;
tf[(0, 'G')] = 9;
tf[(0, 'H')] = 9;
tf[(0, 'I')] = 9;
tf[(0, 'J')] = 9;
tf[(0, 'K')] = 9;
tf[(0, 'L')] = 9;
tf[(0, 'M')] = 9;
tf[(0, 'N')] = 9;
tf[(0, 'O')] = 9;
tf[(0, 'P')] = 9;
tf[(0, 'Q')] = 9;
tf[(0, 'R')] = 9;
tf[(0, 'S')] = 9;
tf[(0, 'T')] = 9;
tf[(0, 'U')] = 9;
tf[(0, 'V')] = 9;
tf[(0, 'W')] = 9;
tf[(0, 'X')] = 9;
tf[(0, 'Y')] = 9;
tf[(0, 'Z')] = 9;
# End tf[(0, L)] = 9;

# Begin tf[(9, L)] = 9;
tf[(9, 'a')] = 9;
tf[(9, 'b')] = 9;
tf[(9, 'c')] = 9;
tf[(9, 'd')] = 9;
tf[(9, 'e')] = 9;
tf[(9, 'f')] = 9;
tf[(9, 'g')] = 9;
tf[(9, 'h')] = 9;
tf[(9, 'i')] = 9;
tf[(9, 'j')] = 9;
tf[(9, 'k')] = 9;
tf[(9, 'l')] = 9;
tf[(9, 'm')] = 9;
tf[(9, 'n')] = 9;
tf[(9, 'o')] = 9;
tf[(9, 'p')] = 9;
tf[(9, 'q')] = 9;
tf[(9, 'r')] = 9;
tf[(9, 's')] = 9;
tf[(9, 't')] = 9;
tf[(9, 'u')] = 9;
tf[(9, 'v')] = 9;
tf[(9, 'w')] = 9;
tf[(9, 'x')] = 9;
tf[(9, 'y')] = 9;
tf[(9, 'z')] = 9;
tf[(9, 'A')] = 9;
tf[(9, 'B')] = 9;
tf[(9, 'C')] = 9;
tf[(9, 'D')] = 9;
tf[(9, 'E')] = 9;
tf[(9, 'F')] = 9;
tf[(9, 'G')] = 9;
tf[(9, 'H')] = 9;
tf[(9, 'I')] = 9;
tf[(9, 'J')] = 9;
tf[(9, 'K')] = 9;
tf[(9, 'L')] = 9;
tf[(9, 'M')] = 9;
tf[(9, 'N')] = 9;
tf[(9, 'O')] = 9;
tf[(9, 'P')] = 9;
tf[(9, 'Q')] = 9;
tf[(9, 'R')] = 9;
tf[(9, 'S')] = 9;
tf[(9, 'T')] = 9;
tf[(9, 'U')] = 9;
tf[(9, 'V')] = 9;
tf[(9, 'W')] = 9;
tf[(9, 'X')] = 9;
tf[(9, 'Y')] = 9;
tf[(9, 'Z')] = 9;
# End tf[(9, L)] = 9;



# Begin tf[(9, D)] = 9;
tf[(9, '0')] = 9;
tf[(9, '1')] = 9;
tf[(9, '2')] = 9;
tf[(9, '3')] = 9;
tf[(9, '4')] = 9;
tf[(9, '5')] = 9;
tf[(9, '6')] = 9;
tf[(9, '7')] = 9;
tf[(9, '8')] = 9;
tf[(9, '9')] = 9;
# End tf[(9, D)] = 9;

tf[(9, '_')] = 9;


tf[(0, '{')] = 10;
tf[(10, '@')] = 10; # aqui o @ substitui ., aceita tudo 
tf[(10, '}')] = 11;


tf[(0, '$')] = 12;

tf[(0, '>')] = 13;
tf[(13, '=')] = 14;

tf[(0, '<')] = 15;
tf[(15, '=')] = 14;
tf[(15, '>')] = 16;
tf[(15, '-')] = 17;

tf[(0, '=')] = 18;

# Begin tf[(0, OP)] = 19;
tf[(0, '+')] = 19;
tf[(0, '-')] = 19;
tf[(0, '*')] = 19;
tf[(0, '/')] = 19;
# End tf[(0, OP)] = 19;

tf[(0, '(')] = 20;
tf[(0, ')')] = 21;
tf[(0, ';')] = 22;

Definicao do alfabeto e estados

In [248]:
states = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};
alphabet = {
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','y','x','z', 
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X', 'Z',
    '0','1','2','3','4','5','6','7','8','9', 
    '+','-','*','/', 
    ' ','{','}','"',';','(',')','.', '>','<','='};
start_state = 0;
accept_states = {1, 3, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};

Inicializacao do automato

In [249]:
d = DFA(states, alphabet, tf, start_state, accept_states);

Teste do automato

In [112]:
inp_program = list("\n");

d.analisys(inp_program);

TypeError: 'list' object is not callable

Definição do Analisador Léxico
----------------------------------------

In [ ]:
class Scanner:
    dfa = None
    file = None
    rowList = None
    def __init__(self):    
        self.file = open('fonte.alg')
        self.rowList = list(self.file)
    def doAnalisys(self):
        for i, row in enumerate(self.rowList):
            charList = list(row)
            for j, char in enumerate(charList):
                print("Caracter: {} Linha: {} Coluna: {}".format(char, i, j))
    
    

In [106]:
list = Scanner().rowList 

In [251]:
list[3]

'\t\tB inteiro;\n'

In [252]:
d.analisys(list[3])

(0, 	) -> 23
(0, 	) -> 23
(0, B) -> 9
debug: adicionando
Lexema: B Token: id Tipo: -
(0,  ) -> 23
(0, i) -> 9
(9, n) -> 9
(9, t) -> 9
(9, e) -> 9
(9, i) -> 9
(9, r) -> 9
(9, o) -> 9
Lexema: inteiro Token: inteiro Tipo: -
(0, ;) -> 22
Lexema: ; Token: PTV Tipo: -
(0, 
) -> 23


In [126]:
import re
x = re.match(r'\n|\t','\n')

In [127]:
print(x)

<re.Match object; span=(0, 1), match='\n'>


In [143]:
a = dict();

# Ignore tab, \n, and spaces
a[(0, ' ')] = 0;
a[(0, '\n')] = 1;
a[(0, '\t')] = 0;

if((0, "\n") in a.keys()):
    print(a[(0, "\n")])
    print('is in transition function')
    
    
    

1
is in transition function
